# Buy Sell

In [1]:
import datetime as dt
import utils as ut
import pandas as pd
import icharts as ic
from functools import cache
from constants import *
import config
from matplotlib import pyplot as plt
import numpy as np
import nse_plot as nplt


TEST_START = dt.datetime.strptime("2023-01-01", "%Y-%m-%d")
TEST_END = dt.datetime.strptime("2023-12-31", "%Y-%m-%d")
INTERVAL = ut.INTERVAL_MIN1
EXCHANGE = ut.EXCHANGE_NSE
SYMBOL = "HDFCBANK"
SYMBOL = "NIFTY 50"
IC_SYMBOL = "NIFTY"

pd.set_option("display.max_colwidth", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.precision", 2)
pd.options.display.float_format = lambda x: '%.2f' % x

def build_date_range(date_start, date_end, symbol):
    date_range = []
    cur_date = date_start
    while cur_date < date_end:
        if cur_date.weekday() not in [5, 6]:
            has_data, _ = ut.has_data(symbol, cur_date, interval=INTERVAL, exchange=EXCHANGE)
            if has_data:
                date_range.append(cur_date)
        cur_date += dt.timedelta(days=1)
    return date_range

all_dates = pd.DataFrame({"trade_date": build_date_range(TEST_START, TEST_END, SYMBOL)})
all_dates_shuffled = all_dates.sample(frac=1, random_state=42)

train_size = int(0.5 * len(all_dates_shuffled))
train_dates = all_dates_shuffled.iloc[:train_size]
test_dates = all_dates_shuffled.iloc[train_size:]
train_dates = all_dates
train_dates = train_dates.sort_values(by="trade_date")
train_dates.set_index("trade_date", inplace=True)
test_dates = test_dates.sort_values(by="trade_date")
test_dates.set_index("trade_date", inplace=True)

def get_intraday_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL, exchange=EXCHANGE)

def get_daily_data(date):
    return ut.get_data(symbol=SYMBOL, date=date, interval=INTERVAL_DAY, exchange=EXCHANGE)

def get_last_trading_day(date):
    return ut.get_last_trading_day(SYMBOL, date, interval=INTERVAL, exchange=ut.EXCHANGE_NSE)

def get_nifty_price(d, t):
    data = ut.get_data(symbol=SYMBOL, date=d, interval=INTERVAL, exchange=EXCHANGE)
    try:
        return data.loc[data.index.time == t].iloc[0].open
    except IndexError:
        return pd.NA
    except AttributeError as e:
        return pd.NA

buy_time = dt.time(hour=15, minute=28)
sell_time = dt.time(hour=9, minute=15)

# train_dates = train_dates.copy()

# train_dates["trade_day_before_expiry"] = train_dates.expiry - pd.Timedelta(days=day_before_expiry)

train_dates["previous_trading_day"] = None
# train_dates["close"] = None
train_dates["previous_trading_day"] = train_dates.apply(lambda row: get_last_trading_day(row.name), axis=1)
train_dates["close"] = train_dates.apply(lambda r: get_nifty_price(r.name.date(), buy_time), axis=1)
train_dates.dropna(inplace=True)
train_dates["yclose"] = train_dates.close.shift(1)
train_dates["is_green"] = train_dates.close > train_dates.yclose

/home/nitin/nse-reports-trade/utils.py:39: DtypeWarning: Columns (14,15) have mixed types. Specify dtype option on import or set low_memory=False.
  scrip_df = pd.read_csv(file_path)


In [4]:
train_dates["trade_qty"] = 0
margin_factor = 8.1
lot_size = 25
max_lot = 10
min_lot = 0
unit_lot = 1

for idx in range(len(train_dates)):
    row = train_dates.iloc[idx]
    if idx == 0:
        continue
    trade_qty = - unit_lot
    if not row.is_green:
        trade_qty = - trade_qty
    prev = train_dates.iloc[idx-1]
    cum_qty = train_dates.iloc[:idx].trade_qty.sum()
    if cum_qty == 0 and trade_qty < 0:
        trade_qty = unit_lot
    if not (min_lot <= (cum_qty + trade_qty) <= max_lot):
        continue
    train_dates.at[row.name, 'trade_qty'] = trade_qty


train_dates["cum_qty"] = train_dates.trade_qty.cumsum()
train_dates["portfolio_value"] = train_dates['cum_qty'] * train_dates['close'] * lot_size
train_dates["trade_value"] = - train_dates['trade_qty'] * train_dates['close'] * lot_size
train_dates["ur_pnl"] = train_dates['trade_value'].cumsum() + train_dates.portfolio_value
print((train_dates["portfolio_value"] / 8).mean())
train_dates
# train_dates["cur_balance"] = train_dates['cum_qty'] * train_dates['close'] * lot_size
# train_dates["rpnl"] = train_dates["ur_pnl"] - train_dates.cum_qty * train_dates.close



251687.75829081633


,previous_trading_day,close,yclose,is_green,trade_qty,cum_qty,portfolio_value,trade_value,ur_pnl
trade_date,,,,,,,,,
2023-01-02,2022-12-30,18213.75,NaN,False,0,0,0.00,0.00,0.00
2023-01-03,2023-01-02,18236.60,18213.75,True,1,1,455915.00,-455915.00,0.00
2023-01-04,2023-01-03,18036.55,18236.60,False,1,2,901827.50,-450913.75,-5001.25
2023-01-05,2023-01-04,18000.60,18036.55,False,1,3,1350045.00,-450015.00,-6798.75
2023-01-06,2023-01-05,17864.25,18000.60,False,1,4,1786425.00,-446606.25,-17025.00
2023-01-09,2023-01-06,18099.65,17864.25,True,-1,3,1357473.75,452491.25,6515.00
2023-01-10,2023-01-09,17921.45,18099.65,False,1,4,1792145.00,-448036.25,-6850.00
2023-01-11,2023-01-10,17891.45,17921.45,False,1,5,2236431.25,-447286.25,-9850.00
2023-01-12,2023-01-11,17866.80,17891.45,False,1,6,2680020.00,-446670.00,-12931.25


## N3gative Gaps